In [ ]:
#!/usr/bin/python3
# coding: utf-8
# sapporo

In [ ]:
import codecs
from datetime import datetime as dt
import datetime
import sys
from jma_csvdl import save_jma_data, parse_jma_csv
import json
import numpy as np
import os
import pandas as pd
import plotly
import plotly.express as px
import plotly.tools as tls
import plotly.graph_objects as go
import plotly.io as pio
import plotly.offline as offline
from plotly.subplots import make_subplots
import sys
from urllib.request import urlretrieve
from cov19utils import create_basic_plot_figure, \
    show_and_clear, moving_average, \
    blank2zero, csv2array, \
    get_twitter, tweet_with_image, \
    get_gpr_predict, FONT_NAME, DT_OFFSET, \
    download_if_needed, json2nparr, code2int, age2int, \
    get_populations, get_os_idx_of_arr, dump_val_in_arr, \
    calc_last1w2w_dif, create_basic_scatter_figure, \
    show_and_save_plotly
from scipy.optimize import curve_fit

In [ ]:
if False:
    #filename="kanagawa.geojson"
    filename="aomori.geojson"
    f=codecs.open(filename, "rb", encoding='utf-8')
    j=json.load(f)
    f.close()
    f=codecs.open(filename.replace("geojson", "min.geojson"), "wb", encoding='utf-8')
    f.write(json.dumps(j, ensure_ascii=False, separators=(',', ':')))
    f.close()

In [ ]:
cov19data = []
data_file = "covid19_data.csv"
# CSVデータを整形する
with codecs.open(data_file, encoding="shift-jis") as f:
    l = f.readline() # 先頭行をスキップ
    while l:
        l = f.readline().replace("\r\n", "")
        arr = l.split(',')
        if len(arr) == 23:
            tracked = blank2zero(arr[20])
            untracked = blank2zero(arr[21])
            unknown_rate = (untracked / max(1.0, (tracked + untracked))) * 100.0
            #print(tracked, untracked, unknown_rate)
            cov19data.append(
                [
                    dt(int(arr[1]), int(arr[2]), int(arr[3])), # date
                    blank2zero(arr[4]),  # tests
                    blank2zero(arr[6]),  # positive
                    blank2zero(arr[19]), # postive rate [%]
                    tracked, # tracked 濃厚接触
                    untracked,  # untracked 濃厚接触以外
                    unknown_rate # 経路不明率
                ]
            )

print("Total: {}".format(len(cov19data)))

In [ ]:
cov19data.append([dt(2020, 11, 7), 0, 187, 0, 0, 0, 0])
pos_rate_np = np.array(cov19data[250:])

In [ ]:
#pos_rate_np
x_offset = 0
xbins = pos_rate_np[:,0]
ybins = pos_rate_np[:,2]
xdays = np.array([i.days for i in (xbins - xbins[0])])
#print(xbins)

In [ ]:
xrange = np.arange(len(xdays[x_offset:]))
(a, b), p0 = curve_fit(lambda t, a, b: a * np.exp(b * t), xrange, ybins[x_offset:], 
                       p0=[1,1], maxfev=1000)
print(a, b)#, p0)
xrange = np.arange(14 + len(xdays[x_offset:]))
y_fit = np.array([a * np.exp(b * i) for i in xrange])
#print(y_fit)
x_pred = np.array(xbins)
for i in np.arange(14):
    x_pred = np.append(x_pred, (xbins[-1] + datetime.timedelta(days=int(1 + i))))
#print(x_pred)

In [ ]:
fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(go.Scatter(x=xbins, y=ybins, mode='markers', name='新規感染者',
                                       marker=dict(size=4)), secondary_y=False)
fig.add_trace(go.Scatter(x=x_pred, y=y_fit, name="a * exp(b * x)",
                         line=dict(width=1)), secondary_y=False)
fig.update_layout(
    xaxis=dict(title='日付', type='date',
               dtick=604800000.0, tickformat="%_m/%-d"),
    yaxis=dict(title='人数', type='log'),
    title='北海道 新型コロナ 新規感染者数/累乗近似 a=%.5f, b=%.5f' % (a, b),
)
show_and_save_plotly(fig, "hokkaido-fit.jpg", js=False)